# HW4
Please do the following in a jupyter notebook and using the data from `data/data.db`

1. Create a jupyter notebook template to do the following - for a given token and date range:
    - calculate the hourly return by timestamp
    - calculate the hourly volatility (you can use the formula we went over in class)
    - calculate maximum drawdown up to that hour for each hour
    - then plot in a 2x2 grid:
        - hourly return
        - hourly vol
        - maximum drawdown
        - close prices
        - color each day on the graph
2. Create a runner notebook using papermill to run the template notebook across all tokens in the database and the last 2 complete weeks in the database.  Summarize the following in a single DataFrame in the runner notebook
    - avg hourly return by run
    - avg volatility by run
    - maximum drawdown over the entire period of each run
    - range of last close price - first close price

# Rachel Cao UCB-Preprogram Course Python HW4 due on 1/3/22 

## Template Notebook

Let's create a cell with all of the necessary imports for this notebook.

In [3]:
import pandas as pd
import numpy as np
import sqlite3
import gspread
import scrapbook as sb

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.dates import date2num, DateFormatter

plt.style.use('ggplot')
plt.tight_layout

%matplotlib inline

## Let's look at the data.db file

In [1]:
%load_ext sql

In [11]:
%%sql sqlite:///data/data.db

SELECT * FROM ohlc WHERE token='BTC' LIMIT 5

Done.


ts,open,high,low,close,volume,volumeUSD,token,chain
2021-11-01 00:00:00,61421.37,61669.14,61239.6,61343.68,256.43386884,15757506.49252652,BTC,BTC
2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.48118498,20445578.51330501,BTC,BTC
2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.25071995,19353904.091577783,BTC,BTC
2021-11-01 03:00:00,61333.17,61457.28,60050.0,60589.06,1059.93135838,64146246.7043883,BTC,BTC
2021-11-01 04:00:00,60590.23,60655.0,59752.92,59971.89,621.41987775,37447441.40394106,BTC,BTC


In [8]:
conn = sqlite3.connect('data/data.db')
cur = conn.cursor()

### Table names are:

In [12]:
for row in cur.execute('SELECT name FROM sqlite_master'):
    print(row)

('ohlc',)
('market_caps',)


### Database schema layout is:

In [13]:
for row in cur.execute('SELECT sql FROM sqlite_master'):
    print(row)

('CREATE TABLE "ohlc" (\n"ts" TIMESTAMP,\n  "open" REAL,\n  "high" REAL,\n  "low" REAL,\n  "close" REAL,\n  "volume" REAL,\n  "volumeUSD" REAL,\n  "token" TEXT,\n  "chain" TEXT\n)',)
('CREATE TABLE market_caps (\n\t"Rank" BIGINT, \n\t"Name" TEXT, \n\t"Symbol" TEXT, \n\t"MarketCap" FLOAT, \n\t"Price" FLOAT, \n\t"VolumeUSD" FLOAT\n)',)


## Find the tokens

In [80]:
%%sql sqlite:///data/data.db

SELECT Distinct token FROM ohlc

Done.


token
BTC
ETH
USDT
SOL
ADA
DOT
AVAX
ATOM
CRV
AAVE


In [85]:
tokens = pd.read_sql("SELECT Distinct token FROM ohlc", connection_string)
s = tokens.squeeze()
s

0      BTC
1      ETH
2     USDT
3      SOL
4      ADA
5      DOT
6     AVAX
7     ATOM
8      CRV
9     AAVE
10    COMP
Name: token, dtype: object

In [44]:
%%sql sqlite:///data/data.db

SELECT * FROM market_caps LIMIT 5

Done.


Rank,Name,Symbol,MarketCap,Price,VolumeUSD
1,Bitcoin,BTC,932695365142.0,49368.85,37198201161.0
2,Ethereum,ETH,498007875686.0,4198.32,25533062707.0
3,Binance Coin,BNB,93038857104.0,557.78,2394235567.0
4,Tether,USDT,75159692181.0,1.0,82054153191.0
5,Solana,SOL,59983315118.0,196.17,3402591975.0


## Parameters cell:

In [45]:
token = 'BTC'
from_date = '2021-11-01'
to_date = '2021-11-05'
connection_string = 'sqlite:///data/data.db'
#events_db = 'crypto_events_database'

## Read the .db file in pandas

In [54]:
prices = pd.read_sql(f"SELECT * FROM ohlc WHERE token='{token}' AND ts >= '{from_date}'AND ts >= '{from_date}' AND ts < '{to_date}'", connection_string)

In [58]:
prices['ts'] = pd.to_datetime(prices['ts'])
prices.set_index('ts', inplace=True)

KeyError: 'ts'

## The volatilty function

In [32]:
def vol_ohlc(df, lookback=10):
    o=df.open
    h=df.high
    c=df.close
    l=df.low
    
    k=0.34 / (1.34 + (lookback + 1) / (lookback-1))
    
    cc=np.log(c/c.shift(1))
    ho=np.log(h/o)
    lo=np.log(l/o)
    co=np.log(c/o)
    oc=np.log(o/c.shift(1))
    oc_sqr=oc**2
    cc_sqr=cc**2
    
    rs= ho*(ho - co) + lo*(lo - co)
    close_vol = cc_sqr.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    open_vol = oc_sqr.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    window_rs = rs.rolling(lookback).sum() * (1.0 / (lookback - 1.0))
    
    result = (open_vol + k * close_vol + (1 - k) * window_rs).apply(np.sqrt) * np.sqrt(252)
    
    result[:lookback -1] = np.nan
    
    return result

In [59]:
prices['volatility'] = vol_ohlc(prices)

ts
2021-11-01 00:00:00         NaN
2021-11-01 01:00:00         NaN
2021-11-01 02:00:00         NaN
2021-11-01 03:00:00         NaN
2021-11-01 04:00:00         NaN
                         ...   
2021-11-04 19:00:00    0.112226
2021-11-04 20:00:00    0.110874
2021-11-04 21:00:00    0.112220
2021-11-04 22:00:00    0.108003
2021-11-04 23:00:00    0.106125
Name: volatility, Length: 96, dtype: float64

## The hourly return

by timestamp

In [61]:
prices

,open,high,low,close,volume,volumeUSD,token,chain,volatility
ts,,,,,,,,,
2021-11-01 00:00:00,61421.37,61669.14,61239.60,61343.68,256.433869,1.575751e+07,BTC,BTC,NaN
2021-11-01 01:00:00,61346.17,61709.82,61171.22,61610.93,332.481185,2.044558e+07,BTC,BTC,NaN
2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC,NaN
2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC,NaN
2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC,NaN
...,...,...,...,...,...,...,...,...,...
2021-11-04 19:00:00,60909.47,61425.00,60754.90,60790.97,620.358241,3.792312e+07,BTC,BTC,0.112226
2021-11-04 20:00:00,60790.96,61321.96,60728.77,61260.90,616.041634,3.761622e+07,BTC,BTC,0.110874
2021-11-04 21:00:00,61271.30,61574.94,61097.56,61370.71,384.366947,2.356648e+07,BTC,BTC,0.112220


In [93]:
prices['hourly_return'] = prices['high']-prices['low']
# not sure if this is the hourly return for %???

## The maximum drawdown


This is calculated from 1 o'clock - 0, 2 o'clock - 0, 3 o'clock - 0, for time set of the day.
Also (trough-peak)/(peak)

In [94]:
def max_drawdown_ohlc(df):
    #this is calculated up to the hour for each day
    
    trough=df.low
    peak=df.high
    for nex in df.low:
        if trough > nex:
            trough = nex
    for nex in df.high:
        if peak < nex:
            peak = nex
    
    result = (trough - peak) / peak
    return result

In [95]:
prices['max_drawdown'] = max_drawdown_ohlc(prices)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## The close prices

In [69]:
prices['close']

ts
2021-11-01 00:00:00    61343.68
2021-11-01 01:00:00    61610.93
2021-11-01 02:00:00    61333.17
2021-11-01 03:00:00    60589.06
2021-11-01 04:00:00    59971.89
                         ...   
2021-11-04 19:00:00    60790.97
2021-11-04 20:00:00    61260.90
2021-11-04 21:00:00    61370.71
2021-11-04 22:00:00    61397.00
2021-11-04 23:00:00    61188.45
Name: close, Length: 96, dtype: float64

## The plotting function with colors
The colors here will be the same rainbow hues for the days of the week

In [70]:
def day_plot(data, ax=None):
    data.plot(ax=ax)
    
    start_of_days = data.between_time('00:00:00', '00:00:00').index.to_list()
    end_of_days = start_of_days[1:] + [data.index[-1]]
    colors = iter(cm.rainbow(np.linspace(0, 1, len(start_of_days))))
    
    for x, y in zip(start_of_days, end_of_days):
        color = next(colors)
        ax.axvspan(x, y, color=color, alpha=0.13)

## Plot the four charts in a 2x2 grid
volatility, hourly return, maximum drawdown, and close prices

In [77]:
%%capture
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20,20))

In [90]:
day_plot(prices[['volatility']], ax=ax[0,0])
day_plot(prices[['hourly_return']], ax=ax[0,1])
day_plot(prices[['max_drawdown']], ax=ax[1,0])
day_plot(prices[['close']], ax=ax[1,1])

KeyError: "None of [Index(['max_drawdown'], dtype='object')] are in the [columns]"

### Glue the scraps with papermill and scrapbook

In [ ]:
sb.glue('token', token)
sb.glue('from_date', from_date)
sb.glue('to_date', to_date)
sb.glue('prices', prices, encoder='pandas')
sb.glue('grid_plot', fig, encoder='display', display=True)